In [2]:
import ase
from ase import Atoms
from ase.visualize import view
from ase.build import bulk, stack
from ase.spacegroup import crystal
from ase.build.tools import niggli_reduce, cut, niggli_reduce_cell
from ase.spacegroup import crystal
from pymatgen.io.cif import CifParser
import numpy as np
import pymatgen as mg
from pymatgen.io.cif import CifParser
from sklearn.neighbors import KDTree
import numpy as np
import csv
import os
import sklearn as skl
from sklearn.neighbors import KDTree

In [3]:
file = "SrTiO3_Pm-3m_80874_80874.cif"
structures = CifParser(file).get_structures()[0]
lattice = structures.lattice
formula = structures.formula.replace(" ", "")
scaled_positions = structures.frac_coords
angles = structures.lattice.angles
abc = structures.lattice.abc
a,b,c = structures.lattice.abc
cell = abc+angles
atom = Atoms(formula, cell=cell, pbc=True, scaled_positions=scaled_positions)
reduced, op = niggli_reduce_cell(np.array(atom.get_cell()))
reduced_atom = Atoms(formula, cell=reduced, pbc=True, scaled_positions=scaled_positions)

# One file

In [43]:
k = int(input('Input k:'))
m = len(structures.sites)
e = np.ceil(np.cbrt(k/m)) #highest integer
if e % 2 == 0:
    e = e + 1

P = [[e,0,0],[0,e,0],[0,0,e]]
supercell = ase.build.make_supercell(reduced_atom,P)
s = 0
Ns = np.ones((m,k))
tree = KDTree(supercell.positions,metric="euclidean")
Ns_1, ind = tree.query(reduced_atom.positions, k=k)

while (Ns == Ns_1).all() == False:
    
    Ns = Ns_1
    
    tree = KDTree(supercell.positions,metric="euclidean")
    Ns_1, ind = tree.query(reduced_atom.positions, k=k)
    
    s = s + 1
    e = e + 2
    P = [[e,0,0],[0,e,0],[0,0,e]]
    
    #print(P)
    #print(e)

    print('No match at step',s)
    
print('Done at step',s+1)
AMD_vector = []
for i in range(k):
    avg = np.mean(Ns_1[:,i])
    AMD_vector.append(avg)

#print(AMD_vector)
print(Ns_1)

Input k:5
No match at step 1
Done at step 2
[[0.00000000e+00 2.75842355e+00 2.75842355e+00 2.75842355e+00
  3.37836510e+00]
 [3.84592537e-16 1.95050000e+00 1.95050000e+00 1.95050000e+00
  3.37836510e+00]
 [3.14018492e-16 1.95050000e+00 2.75842355e+00 2.75842355e+00
  2.75842355e+00]
 [3.14018492e-16 1.95050000e+00 2.75842355e+00 2.75842355e+00
  2.75842355e+00]
 [3.14018492e-16 1.95050000e+00 2.75842355e+00 2.75842355e+00
  2.75842355e+00]]
